Import the pandas library

In [2]:
import pandas as pd
!conda install -c conda-forge geopy --yes 
from bs4 import BeautifulSoup
import requests

In [3]:
#wiki=pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M",header=0)
#df_toranto=pd.DataFrame(wiki[:1])
#page="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
#wikitables = pd.read_html(page, index_col=0, attrs={"class":"wikitable"}, header=0) 
#wikitables
##wikitables[0].to_csv("example.csv")
#df=pd.read_csv("example.csv")
#df.head(5)
#df=df[df["Neighbourhood"]!="Not assigned"].reset_index(drop=True)
#df
   

Install Beautifuulsoup 

In [4]:
!pip install beautifulsoup4

distributed 1.21.8 requires msgpack, which is not installed.


import requests and beautifulsoup

get the HTML page using requests.get() and pass it to beautifulsoup constructor to parse it

In [7]:
URL="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
results=requests.get(URL).text
soup=BeautifulSoup(results,"lxml")

get the borough and Neighborhood details add it to a dataframe

In [36]:
table_req=soup.find("table",{"class":"wikitable sortable"})
data=[]
table_body=table_req.find("tbody")
table_body
rows = table_body.find_all("tr")
for row in rows:
    cols=row.find_all("td")
    cols=[ele.text.strip() for ele in cols]
    data.append([ele for ele in cols if ele])
    
#data
df=pd.DataFrame(data[1:])
df=df[df[1] != "Not assigned"].reset_index(drop=True)
df.columns=["PostCode","Borough","Neighborhood"]

In [41]:
df=df.groupby(['PostCode','Borough'])['Neighborhood'].apply(','.join).reset_index()


In [74]:
df.shape

(103, 5)

In [65]:
def get_latlong(address):
    #address="North York, Toronto, Ontario"
    lat=[]
    long=[]
    geolocator=Nominatim()
    for a in address:
        lat_long =None
        while (lat_long is None):
            lat_long=geolocator.geocode(address)
            lat.append(lat_long.latitude)
            long.append(lat_long.longitude)
            
    return lat,long
    

Using the provided file since The geocoder method is giving server error.

In [ ]:
df1=pd.read_csv("http://cocl.us/Geospatial_data")

In [87]:
df["Latitude"]=df1["Latitude"]
df["Longitude"]=df1["Longitude"]

In [106]:
df.head()

,PostCode,Borough,Neighborhood,Latitude,Londitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,-79.239476


#### use geopy library to get the latitude and longitude values of Toronto

In [81]:
geolocator=Nominatim()
location=geolocator.geocode("Toronto , Ontario, Canada")
lat=location.latitude
long=location.longitude

/home/jupyterlab/conda/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


In [85]:
#!conda install -c conda-forge folium=0.5.0 --yes 
#import folium

#### Create a map of totonto with neighborhoods superimposed on to

In [103]:
map_toronto = folium.Map(location=[lat, long], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
#map_toronto

In [109]:
df_scar=df[df["Borough"]=="Scarborough"].reset_index(drop = True)
df_scar.shape

(17, 6)

In [110]:
address = 'Scarborough, toronto, Ontario'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude, longitude))

/home/jupyterlab/conda/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of Scarborough are 43.773077, -79.257774.


In [113]:
map_scar = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df_scar['Latitude'], df_scar['Longitude'], df_scar['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_scar)  
    
map_scar

'Guildwood,Morningside,West Hill'